<a href="https://colab.research.google.com/github/ValeriaYashan/Machine-learning/blob/main/Machine_learning_predicci%C3%B3n_de_acciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 A COMPLETAR


El modelo de Random Forest aplicado al análisis fundamental de acciones intentaría predecir si el precio de una acción aumentará o disminuirá en el futuro en función de las características financieras históricas de la empresa. Estas características podrían incluir datos como el precio de cierre de la acción, el volumen de negociación, los ingresos de la empresa, el crecimiento de las ganancias, los ratios financieros, entre otros.

Una vez entrenado el modelo con datos históricos que incluyen estas características y la dirección futura del precio de las acciones, el modelo puede ser utilizado para hacer predicciones sobre nuevos datos. Por ejemplo, si proporcionamos al modelo información sobre el precio de cierre de una acción, el volumen de negociación y otros datos financieros relevantes para un determinado día, el modelo puede predecir si el precio de esa acción aumentará o disminuirá en el próximo período de tiempo.

Estas predicciones pueden ser útiles para los inversores y analistas financieros que desean tomar decisiones informadas sobre qué acciones comprar, vender o mantener en sus carteras. Sin embargo, es importante tener en cuenta que ningún modelo de machine learning es perfecto y siempre existe cierto nivel de incertidumbre asociado con las predicciones. Es por eso que es crucial realizar una evaluación cuidadosa del rendimiento del modelo y considerar otros factores antes de tomar decisiones de inversión basadas únicamente en las predicciones del modelo.

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import files

# Subir el archivo "acciones" desde tu computadora
uploaded = files.upload()


# Verificar que el archivo se haya cargado correctamente
for filename in uploaded.keys():
    print('El archivo "{}" se ha cargado correctamente.'.format(filename))

# Cargar el archivo CSV de acciones
df_acciones_con_nombres = pd.read_csv(filename)




Saving Acciones_con_nombres.csv to Acciones_con_nombres (2).csv
El archivo "Acciones_con_nombres (2).csv" se ha cargado correctamente.


In [ ]:
# Mostrar las primeras filas del DataFrame
print(df_acciones_con_nombres.head())

# Verificar nombres de columnas
print("Nombres de columnas:")
print(df_acciones_con_nombres.columns)

# Verificar tipos de datos
print("\nTipos de datos:")
print(df_acciones_con_nombres.dtypes)

# Verificar valores nulos o faltantes
print("\nValores nulos:")
print(df_acciones_con_nombres.isnull().sum())

# Verificar duplicados
print("\nDuplicados:")
print(df_acciones_con_nombres.duplicated().sum())

# Normalización de datos (si es necesario)
# Ejemplo de normalización min-max
# df['columna_normalizada'] = (df_acciones['columna'] - df_acciones['columna'].min()) / (df_acciones['columna'].max() - df_acciones['columna'].min())



         Fecha     Simbolo  Apertura    Máximo  Mínimo       Cierre  \
0  Instrumento       Fecha       NaN    Precio   Var %  Var Anual %   
1        MOLA5  24/10/2019       NaN    219,50  -24,31        -0,20   
2         MOLA  09/02/2024       NaN  26700,00    6,49         8,00   
3        MOLI5  21/11/2019       NaN     35,00    0,00       -23,90   
4           CL  08/04/2022       NaN   1000,00    0,00          NaN   

   Cierre_Anterior Variación               Volumen  
0              NaN     TIR %  Mod. Duration (años)  
1              NaN       NaN                   NaN  
2              NaN       NaN                   NaN  
3              NaN       NaN                   NaN  
4              NaN       NaN                   NaN  
Nombres de columnas:
Index(['Fecha', 'Simbolo', 'Apertura', 'Máximo', 'Mínimo', 'Cierre',
       'Cierre_Anterior', 'Variación', 'Volumen'],
      dtype='object')

Tipos de datos:
Fecha               object
Simbolo             object
Apertura           fl

In [ ]:
# Crear el DataFrame con los datos proporcionados
data = {
    'Fecha': ['24/10/2019', '09/02/2024', '21/11/2019', '08/04/2022', '09/02/2024'],
    'Simbolo': ['MOLA5', 'MOLA', 'MOLI5', 'CL', 'PAMP'],
    'Apertura': [None, None, None, None, None],
    'Máximo': [219.50, 26700.00, 35.00, 1000.00, 2198.45],
    'Mínimo': [-24.31, 6.49, 0.00, 0.00, 0.04],
    'Cierre': [-0.20, 8.00, -23.90, None, 3.57],
    'Cierre_Anterior': [None, None, None, None, None],
    'Variación': [None, None, None, None, None],
    'Volumen': [None, None, None, None, None]
}

# Crear el DataFrame
acciones_con_nombres = pd.DataFrame(data)

# Eliminar la primera fila de descripción
acciones_con_nombres = acciones_con_nombres.iloc[1:]

# Convertir las columnas numéricas al tipo de datos adecuado
columnas_numericas = ['Apertura', 'Máximo', 'Mínimo', 'Cierre', 'Cierre_Anterior', 'Variación', 'Volumen']
acciones_con_nombres[columnas_numericas] = acciones_con_nombres[columnas_numericas].apply(pd.to_numeric, errors='coerce')

# Verificar los primeros registros y las columnas
print(acciones_con_nombres.head())
print(acciones_con_nombres.info())

        Fecha Simbolo  Apertura    Máximo  Mínimo  Cierre  Cierre_Anterior  \
1  09/02/2024    MOLA       NaN  26700.00    6.49    8.00              NaN   
2  21/11/2019   MOLI5       NaN     35.00    0.00  -23.90              NaN   
3  08/04/2022      CL       NaN   1000.00    0.00     NaN              NaN   
4  09/02/2024    PAMP       NaN   2198.45    0.04    3.57              NaN   

   Variación  Volumen  
1        NaN      NaN  
2        NaN      NaN  
3        NaN      NaN  
4        NaN      NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 1 to 4
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fecha            4 non-null      object 
 1   Simbolo          4 non-null      object 
 2   Apertura         0 non-null      float64
 3   Máximo           4 non-null      float64
 4   Mínimo           4 non-null      float64
 5   Cierre           3 non-null      float64
 6   Cierre_Anterior 

In [ ]:
# Definir las características (features) y la variable objetivo (target)
X = acciones_con_nombres.drop(columns=['Fecha', 'Simbolo', 'Cierre'])
y = acciones_con_nombres['Cierre']

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Cargar los datos y dividirlos en características (X) y variable objetivo (y)
# Aquí se asume que ya tienes cargados y divididos los datos de entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar el imputador para rellenar los valores NaN con la media de cada columna
imputer = SimpleImputer(strategy='mean')

# Imputar los valores faltantes en el conjunto de entrenamiento
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)

# Eliminar las filas con valores NaN del conjunto de entrenamiento
X_train_cleaned = X_train_imputed.dropna()
y_train_cleaned = y_train[X_train_cleaned.index]

# Obtener las columnas presentes en los datos imputados
columns_in_imputed_data = X_train_cleaned.columns

# Filtrar el conjunto de entrenamiento original para incluir solo las columnas presentes en los datos imputados
X_train_filtered = X_train[columns_in_imputed_data]

# Inicializar y entrenar el modelo de regresión con los datos limpios
reg_model = RandomForestRegressor(n_estimators=100, random_state=42)
reg_model.fit(X_train_filtered, y_train_cleaned)

# Imprimir el puntaje de entrenamiento del modelo
print("Puntaje de entrenamiento del modelo de regresión:", reg_model.score(X_train_filtered, y_train_cleaned))

ValueError: Shape of passed values is (3, 2), indices imply (3, 6)